# [CityBikes](https://citybik.es/)

### Send a request to CityBikes for the city of your choice. 

In [1]:
import requests
import json
import pandas as pd

In [32]:
# Call fields, name, href, and location to figure out city endpoint
url = 'http://api.citybik.es/v2/networks'
response = requests.get(url)
print(response)
networks = response.json()

<Response [200]>


Parsing through and determining keys in json dictionary

networks.keys() <br> 
dict_keys(['networks'])

networks['networks'][0].keys() <br>
dict_keys(['company', 'href', 'id', 'location', 'name'])

networks['networks'][0]['location'].keys()<br>
dict_keys(['city', 'country', 'latitude', 'longitude'])


In [33]:
networks['networks'][688]

{'company': ['BCycle, LLC'],
 'gbfs_href': 'https://gbfs.bcycle.com/bcycle_redding/gbfs.json',
 'href': '/v2/networks/redding',
 'id': 'redding',
 'location': {'city': 'Redding, CA',
  'country': 'US',
  'latitude': 40.5875,
  'longitude': -122.386},
 'name': 'Redding BCycle'}

In [46]:
# Finding index of Redding, CA in networks
for i in range(len(networks['networks'])):
    if 'Redding' in networks['networks'][i]['location']['city']:
        index = i

index

688

In [47]:
href = networks['networks'][index]['href']

In [28]:
url = fr'http://api.citybik.es{href}'
response = requests.get(url)
results = response.json()

In [ ]:
with open('../data/ReddingCityBike.json', 'w') as f:
    json.dump(results, f)

### Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

Keys for Reddings Results

results.keys() <br>
dict_keys(['networks'])

results['network'].keys() <br>
dict_keys(['company', 'gbfs_href', 'href', 'id', 'location', 'name', 'stations'])

results['network']['location'].keys()
dict_keys(['city', 'country', 'latitude', 'longitude'])

results['network']['stations'][0].keys()
dict_keys(['empty_slots', 'extra', 'free_bikes', 'id', 'latitude', 'longitude', 'name', 'timestamp'])



In [59]:
print(results['network']['company'])
print(results['network']['gbfs_href'])
print(results['network']['href'])
print(results['network']['id'])
print(results['network']['location'])
print(results['network']['name'])


['BCycle, LLC']
https://gbfs.bcycle.com/bcycle_redding/gbfs.json
/v2/networks/redding
redding
{'city': 'Redding, CA', 'country': 'US', 'latitude': 40.5875, 'longitude': -122.386}
Redding BCycle


In [ ]:
results['network']['stations'][0]

{'empty_slots': 2,
 'extra': {'address': 'Market Street & Gold Street',
  'ebikes': 2,
  'has_ebikes': True,
  'last_updated': 1716266776,
  'normal_bikes': 0,
  'rental_uris': {'android': 'https://www.bcycle.com/applink?system_id=bcycle_redding&station_id=bcycle_redding_7427&platform=android',
   'ios': 'https://www.bcycle.com/applink?system_id=bcycle_redding&station_id=bcycle_redding_7427&platform=iOS'},
  'renting': 1,
  'returning': 1,
  'uid': 'bcycle_redding_7427'},
 'free_bikes': 2,
 'id': 'bb7665847eda4d3b5654ef23584bba2f',
 'latitude': 40.58002,
 'longitude': -122.3889,
 'name': 'Market and Gold Street',
 'timestamp': '2024-05-21T04:46:17.148000Z'}

In [48]:
len(results['network']['stations'])

22

In [55]:
pd.json_normalize(results['network']['stations']).head()

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.normal_bikes,extra.rental_uris.android,extra.rental_uris.ios,extra.renting,extra.returning,extra.uid
0,2,2,bb7665847eda4d3b5654ef23584bba2f,40.58002,-122.38890,Market and Gold Street,2024-05-21T04:46:17.148000Z,Market Street & Gold Street,2,True,1716266776,0,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_redding_7427
1,3,2,3bb59ffd3799adff3f475e8d3985532f,40.58352,-122.39042,Market Center South,2024-05-21T04:46:17.152000Z,Market Street & Yuba Street,2,True,1716266776,0,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_redding_7426
2,2,3,08aae6d19bdfffb93561437caf892f36,40.58435,-122.39067,Market Center North,2024-05-21T04:46:17.153000Z,1371 Butte Street,3,True,1716266776,0,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_redding_7425
3,3,2,dd5c0ef4079f9e920b3cdba5c3ca167c,40.59502,-122.40150,Diestelhorst Bridge,2024-05-21T04:46:17.160000Z,Benton Drive,2,True,1716266776,0,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_redding_7424
4,3,3,64a7ac84a24f102734b87abc4ba71af0,40.57337,-122.38177,Redding City Hall,2024-05-21T04:46:17.161000Z,777 Cypress Ave,3,True,1716266776,0,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_redding_7423


### Put your parsed results into a DataFrame.

In [62]:
ReddingStations = pd.json_normalize(results['network']['stations'])

In [66]:
ReddingStations.head()

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra.address,extra.ebikes,extra.has_ebikes,extra.last_updated,extra.normal_bikes,extra.rental_uris.android,extra.rental_uris.ios,extra.renting,extra.returning,extra.uid
0,2,2,bb7665847eda4d3b5654ef23584bba2f,40.58002,-122.38890,Market and Gold Street,2024-05-21T04:46:17.148000Z,Market Street & Gold Street,2,True,1716266776,0,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_redding_7427
1,3,2,3bb59ffd3799adff3f475e8d3985532f,40.58352,-122.39042,Market Center South,2024-05-21T04:46:17.152000Z,Market Street & Yuba Street,2,True,1716266776,0,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_redding_7426
2,2,3,08aae6d19bdfffb93561437caf892f36,40.58435,-122.39067,Market Center North,2024-05-21T04:46:17.153000Z,1371 Butte Street,3,True,1716266776,0,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_redding_7425
3,3,2,dd5c0ef4079f9e920b3cdba5c3ca167c,40.59502,-122.40150,Diestelhorst Bridge,2024-05-21T04:46:17.160000Z,Benton Drive,2,True,1716266776,0,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_redding_7424
4,3,3,64a7ac84a24f102734b87abc4ba71af0,40.57337,-122.38177,Redding City Hall,2024-05-21T04:46:17.161000Z,777 Cypress Ave,3,True,1716266776,0,https://www.bcycle.com/applink?system_id=bcycl...,https://www.bcycle.com/applink?system_id=bcycl...,1,1,bcycle_redding_7423


In [67]:
ReddingStations.to_csv('../data/ReddingBikesDF.csv',index=False)